<a href="https://colab.research.google.com/github/crunchdomo/llm_conversation/blob/main/test_base_openai_baselne.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
import openai
import json
from datetime import datetime
import re
import random

# Initialize OpenAI client
client = openai.OpenAI(api_key="")  # Replace with your actual API key

def parse_steps(instructions):
    """Parse instructions into individual steps"""
    steps = re.split(r'\n\s*\d+\.\s+', instructions.strip())
    return [f"{i+1}. {s.strip()}" for i, s in enumerate(steps) if s.strip()]

def cook_recipe(recipe_data, automated_inputs, model="gpt-4-turbo", conversation_type="default"):
    # Parse steps and initialize tracking
    steps = parse_steps(recipe_data['instructions'])

    # Define conversation templates
    conversation_templates = {
        "combined": {
            1: [
                "Can I use sea salt instead of kosher salt?",  # Substitution
                "Why do we let the chicken sit at room temperature?"  # Technique
            ],
            3: [
                "Can I substitute rosemary with thyme?",  # Substitution
                "What's the purpose of tossing the squash with herbs?"  # Technique
            ]
        },
        "substitution_questions": {
            1: ["Can I use sea salt instead of kosher salt?"],
            3: ["Can I substitute rosemary with thyme?"]
        },
        "technique_clarification": {
            1: ["Why do we let the chicken sit at room temperature?"],
            3: ["What's the purpose of tossing the squash with herbs?"]
        }
    }

    # Initialize system prompt
    chat = [{
        "role": "system",
        "content": f"""You are a master chef guiding through: {recipe_data['title']}
        - Begin each step with "STEP: [NUMBER]"
        - Use metric measurements
        - End each step with "AWAITING CONFIRMATION"
        - After final step, say "CONVO-COMPLETE"
        - Answer questions about specific steps using their numbers"""
    }]

    # Process steps with automated inputs
    step_index = 0
    input_index = 0

    while step_index < len(steps) and input_index < len(automated_inputs):
        # Send current step
        chat.append({
            "role": "user",
            "content": f"Present step {step_index+1} clearly: {steps[step_index]}"
        })

        # Get chef's response
        response = client.chat.completions.create(
            model=model,
            messages=chat,
            max_tokens=512,
            temperature=0.7
        )
        chef_response = response.choices[0].message.content
        print(f"\nChef: {chef_response}")
        chat.append({"role": "assistant", "content": chef_response})

        # Handle automated input
        user_input = automated_inputs[input_index]
        print(f"\nTrainee: {user_input}")
        chat.append({"role": "user", "content": user_input})
        input_index += 1

        # Inject template questions
        current_step = step_index + 1
        if conversation_type in conversation_templates:
            template = conversation_templates[conversation_type]
            if current_step in template:
                for question in template[current_step]:
                    print(f"\n[Template Question] Trainee: {question}")
                    chat.append({"role": "user", "content": question})

                    # Get answer
                    response = client.chat.completions.create(
                        model=model,
                        messages=chat,
                        max_tokens=256,
                        temperature=0.7
                    )
                    answer = response.choices[0].message.content
                    print(f"\nChef: {answer}")
                    chat.append({"role": "assistant", "content": answer})

        step_index += 1

    save_conversation(chat, f"cooking_session_{conversation_type}_{model}.json")
    return chat


def save_conversation(chat_history, filename=None):
    if not filename:
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        filename = f"cooking_session_{timestamp}.json"

    with open(filename, 'w') as f:
        json.dump(chat_history, f, indent=2)
    print(f"Saved to {filename}")

# Recipe data structure
recipe = {
    "title": "Miso-Butter Roast Chicken With Acorn Squash Panzanella",
    "ingredients": [
        '1 (3½–4-lb.) whole chicken', '2¾ tsp. kosher salt, divided, plus more',
        '2 small acorn squash (about 3 lb. total)', '2 Tbsp. finely chopped sage',
        '1 Tbsp. finely chopped rosemary', '6 Tbsp. unsalted butter, melted, plus 3 Tbsp. room temperature',
        '¼ tsp. ground allspice', 'Pinch of crushed red pepper flakes', 'Freshly ground black pepper',
        '⅓ loaf good-quality sturdy white bread, torn into 1" pieces (about 2½ cups)',
        '2 medium apples (such as Gala or Pink Lady; about 14 oz. total), cored, cut into 1" pieces',
        '2 Tbsp. extra-virgin olive oil', '½ small red onion, thinly sliced',
        '3 Tbsp. apple cider vinegar', '1 Tbsp. white miso', '¼ cup all-purpose flour',
        '2 Tbsp. unsalted butter, room temperature', '¼ cup dry white wine',
        '2 cups unsalted chicken broth', '2 tsp. white miso', 'Kosher salt, freshly ground pepper'
    ],
    "instructions": """
        1. Pat chicken dry with paper towels, season all over with 2 tsp. salt, and tie legs together with kitchen twine. Let sit at room temperature 1 hour.

        2. Meanwhile, halve squash and scoop out seeds. Run a vegetable peeler along ridges of squash halves to remove skin. Cut each half into ½""-thick wedges; arrange on a rimmed baking sheet.

        3. Combine sage, rosemary, and 6 Tbsp. melted butter in a large bowl; pour half of mixture over squash on baking sheet. Sprinkle squash with allspice, red pepper flakes, and ½ tsp. salt and season with black pepper; toss to coat.

        4. Add bread, apples, oil, and ¼ tsp. salt to remaining herb butter in bowl; season with black pepper and toss to combine. Set aside.
        Place onion and vinegar in a small bowl; season with salt and toss to coat. Let sit, tossing occasionally, until ready to serve.

        5. Place a rack in middle and lower third of oven; preheat to 425°F. Mix miso and 3 Tbsp. room-temperature butter in a small bowl until smooth. Pat chicken dry with paper towels, then rub or brush all over with miso butter. Place chicken in a large cast-iron skillet and roast on middle rack until an instant-read thermometer inserted into the thickest part of breast registers 155°F, 50–60 minutes. (Temperature will climb to 165°F while chicken rests.) Let chicken rest in skillet at least 5 minutes, then transfer to a plate; reserve skillet.

        6. Meanwhile, roast squash on lower rack until mostly tender, about 25 minutes. Remove from oven and scatter reserved bread mixture over, spreading into as even a layer as you can manage. Return to oven and roast until bread is golden brown and crisp and apples are tender, about 15 minutes. Remove from oven, drain pickled onions, and toss to combine. Transfer to a serving dish.

        7. Using your fingers, mash flour and butter in a small bowl to combine.

        8. Set reserved skillet with chicken drippings over medium heat. You should have about ¼ cup, but a little over or under is all good. (If you have significantly more, drain off and set excess aside.) Add wine and cook, stirring often and scraping up any browned bits with a wooden spoon, until bits are loosened and wine is reduced by about half (you should be able to smell the wine), about 2 minutes. Add butter mixture; cook, stirring often, until a smooth paste forms, about 2 minutes. Add broth and any reserved drippings and cook, stirring constantly, until combined and thickened, 6–8 minutes. Remove from heat and stir in miso. Taste and season with salt and black pepper.

        9. Serve chicken with gravy and squash panzanella alongside.
            """
}

# Predefined automated trainee inputs
automated_inputs = [
    "Got it! Ready for step 1.",
    "Done Continue.",
    "Done Continue.",
    "Done Continue.",
    "Done Continue.",
    "Done Continue.",
    "Done Continue.",
    "Done Continue.",
    "Done Continue.",
    "Done Continue.",
    "Done Continue.",
    "exit"
]

# Start cooking session with automated inputs
models = ["gpt-4-turbo", "gpt-3.5-turbo"]
templates = ["combined"]

for model in models:
    for template in templates:
        cook_recipe(recipe, automated_inputs, model=model, conversation_type=template)



Chef: STEP: 1
Pat the chicken dry with paper towels, season it all over with 10 grams of salt, and tie the legs together with kitchen twine. Let it sit at room temperature for 1 hour.
AWAITING CONFIRMATION

Trainee: Got it! Ready for step 1.

[Template Question] Trainee: Can I use sea salt instead of kosher salt?

Chef: Yes, you can use sea salt instead of kosher salt for seasoning the chicken. Just keep in mind that sea salt can sometimes be finer than kosher salt, so you might want to use a little less to avoid over-salting. Adjust according to your taste preference.

[Template Question] Trainee: Why do we let the chicken sit at room temperature?

Chef: Letting the chicken sit at room temperature for an hour as described in Step 1 helps in a couple of ways. First, it allows the salt to season the chicken more deeply, enhancing its flavor throughout. Second, bringing the chicken closer to room temperature ensures more even cooking when roasted. Cold meat straight from the refrigerato